In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

### Configuration

In [16]:
batch_size = 64 #Batch size for trainning
epochs = 100 #Number of epohs to train for 
latent_dim = 256 #Latent dimensionality of the encoding space.
num_samples = 10000 #number of sampples to train
data_path = "fra-eng/fra.txt"

### Prepare the data

In [17]:
#vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")



In [18]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" charachter
    # for the targets, and "\n" as "end sequence" charachter.
    
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            

In [19]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [20]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)   


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [21]:
input_token_index = dict([char, i] for i, char in enumerate(input_characters))
target_token_index = dict([char, i] for i, char in enumerate(target_characters))

In [22]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [23]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

### Build the model

In [24]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Train the model

In [25]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

model.save("s2s")

Epoch 1/100
125/125 [==============================] - 46s 321ms/step - loss: 1.2276 - accuracy: 0.7309 - val_loss: 1.1229 - val_accuracy: 0.7081
Epoch 2/100
125/125 [==============================] - 43s 347ms/step - loss: 0.9408 - accuracy: 0.7481 - val_loss: 0.9991 - val_accuracy: 0.7101
Epoch 3/100
125/125 [==============================] - 39s 316ms/step - loss: 0.8482 - accuracy: 0.7656 - val_loss: 0.9057 - val_accuracy: 0.7437
Epoch 4/100
125/125 [==============================] - 35s 278ms/step - loss: 0.7581 - accuracy: 0.7896 - val_loss: 0.8269 - val_accuracy: 0.7664
Epoch 5/100
125/125 [==============================] - 35s 283ms/step - loss: 0.6784 - accuracy: 0.8062 - val_loss: 0.7703 - val_accuracy: 0.7715
Epoch 6/100
125/125 [==============================] - 39s 313ms/step - loss: 0.6341 - accuracy: 0.8160 - val_loss: 0.7164 - val_accuracy: 0.7907
Epoch 7/100
125/125 [==============================] - 35s 276ms/step - loss: 0.6044 - accuracy: 0.8228 - val_loss: 0.6895 -

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets
